# Packages

In [1]:
using CSV, DataFrames, PyCall, Plots
using LinearAlgebra
using TimeSeries
using TimeSeriesResampler
#using Impute
using Statistics
#using RollingFunctions
using Dates
using Pandas
using Downloads
using DataFrames
using MarketData

# Include jl files

In [2]:
include("DataStructures.jl");

# Import Tick data

In [3]:
dir = "https://raw.githubusercontent.com/risk-labratory/data/main/"
url = string(dir, "IVE_2020.csv")
p = Downloads.download(url, "Data.csv")

"Data.csv"

In [4]:
dataframe = CSV.read("Data.csv", DataFrames.DataFrame, header=0)
delete!(dataframe, 1)
DataFrames.rename!(dataframe, :Column1 => :dates,:Column2 => :price, :Column3 => :bid, :Column4 => :ask, :Column5 => :size)
dataframe.dates = DateTime.(dataframe.dates, "yyyy-mm-dd HH:MM:SS")
dataframe.price = parse.(Float64, dataframe.price)
dataframe.bid = parse.(Float64, dataframe.bid)
dataframe.ask = parse.(Float64, dataframe.ask)
dataframe.size = parse.(Float64, dataframe.size)
dataframe = dataframe[(hour.(dataframe.dates) .>= 9 .&& hour.(dataframe.dates) .< 16), :]
first(dataframe, 15)

,dates,price,bid,ask,size
,DateTime,Float64,Float64,Float64,Float64
1,2020-01-02T09:30:00,130.68,130.59,130.6,20625.0
2,2020-01-02T09:30:01,130.5,130.5,130.77,200.0
3,2020-01-02T09:30:04,130.53,130.52,130.78,100.0
4,2020-01-02T09:30:04,130.55,130.52,130.78,100.0
5,2020-01-02T09:30:04,130.53,130.52,130.78,200.0
6,2020-01-02T09:30:04,130.54,130.52,130.78,100.0
7,2020-01-02T09:30:04,130.53,130.52,130.78,300.0
8,2020-01-02T09:30:04,130.52,130.52,130.78,100.0
9,2020-01-02T09:30:04,130.52,130.52,130.78,200.0


In [5]:
ohlcvTime = timeBar(dataframe, 30)
first(ohlcvTime, 10)

,dates,Open,High,Low,Close,Volume,ValueOfTrades
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-02T09:30:00,130.68,130.78,130.37,130.43,164048.0,2.14222e7
2,2020-01-02T10:00:00,130.4,130.4,130.022,130.056,75227.0,9.79016e6
3,2020-01-02T10:30:00,130.08,130.2,129.971,130.2,91006.0,1.1838e7
4,2020-01-02T11:00:00,130.205,130.209,130.08,130.13,65182.0,8.48291e6
5,2020-01-02T11:30:00,130.083,130.083,129.938,130.03,84253.0,1.0954e7
6,2020-01-02T12:00:00,130.022,130.118,129.98,130.073,59420.0,7.7284e6
7,2020-01-02T12:30:00,130.095,130.135,130.042,130.077,152349.0,1.98183e7
8,2020-01-02T13:00:00,130.07,130.07,129.957,129.957,16861.0,2.19186e6
9,2020-01-02T13:30:00,129.98,130.02,129.94,129.978,64199.0,8.34389e6


In [6]:
timeBarLen = size(ohlcvTime)[1]
ohlcvTick = tickBar(dataframe, nothing, timeBarLen)
first(ohlcvTick, 10)

,dates,Open,High,Low,Close,Volume,ValueOfTrades
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-02T09:30:00,130.68,130.78,130.5,130.61,47574.0,6.21517e6
2,2020-01-02T09:33:10,130.61,130.67,130.45,130.53,69207.0,9.03804e6
3,2020-01-02T09:42:59,130.515,130.62,130.37,130.41,44946.0,5.86627e6
4,2020-01-02T09:56:38,130.403,130.46,130.065,130.118,38961.0,5.07294e6
5,2020-01-02T10:18:03,130.118,130.16,130.022,130.072,48107.0,6.25818e6
6,2020-01-02T10:37:04,130.075,130.078,129.971,130.035,24254.0,3.15398e6
7,2020-01-02T10:52:14,130.032,130.209,129.995,129.995,125485.0,1.63281e7
8,2020-01-02T11:35:24,129.96,130.109,129.938,130.063,120848.0,1.57131e7
9,2020-01-02T12:15:03,130.05,130.135,130.036,130.08,113645.0,1.47834e7


In [7]:
timeBarLen = size(ohlcvTime)[1]
ohlcvVolume = volumeBar(dataframe, nothing, timeBarLen)
first(ohlcvVolume, 10)

,dates,Open,High,Low,Close,Volume,ValueOfTrades
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-02T09:30:00,130.68,130.78,130.5,130.59,36253.0,4.73681e6
2,2020-01-02T09:32:50,130.57,130.62,130.557,130.62,12521.0,1.63509e6
3,2020-01-02T09:33:22,130.61,130.67,130.48,130.54,61782.0,8.06882e6
4,2020-01-02T09:37:19,130.54,130.62,130.45,130.58,37265.0,4.86451e6
5,2020-01-02T09:48:02,130.58,130.595,130.095,130.095,37144.0,4.84167e6
6,2020-01-02T10:16:44,130.095,130.16,130.065,130.1,36842.0,4.79357e6
7,2020-01-02T10:20:38,130.116,130.134,129.99,129.99,36861.0,4.79422e6
8,2020-01-02T10:38:52,129.988,130.137,129.971,130.11,21133.0,2.74811e6
9,2020-01-02T10:56:49,130.1,130.209,130.1,130.198,53176.0,6.91857e6


In [8]:
ohlcvDollar = dollarBar(dataframe, nothing, timeBarLen)
first(ohlcvDollar, 10)

,dates,Open,High,Low,Close,Volume,ValueOfTrades
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-02T09:30:00,130.68,130.78,130.5,130.74,31371.0,4.09912e6
2,2020-01-02T09:30:12,130.67,130.705,130.557,130.62,17403.0,2.27279e6
3,2020-01-02T09:33:22,130.61,130.67,130.45,130.536,73744.0,9.63018e6
4,2020-01-02T09:44:31,130.54,130.62,130.37,130.37,34400.0,4.49026e6
5,2020-01-02T09:55:33,130.379,130.46,130.095,130.096,30750.0,4.00621e6
6,2020-01-02T10:16:44,130.096,130.16,130.065,130.12,28967.0,3.76904e6
7,2020-01-02T10:20:23,130.1,130.134,130.022,130.066,34782.0,4.52405e6
8,2020-01-02T10:37:10,130.066,130.137,129.971,130.11,28384.0,3.69116e6
9,2020-01-02T10:56:49,130.1,130.209,130.1,130.175,60491.0,7.87075e6


# Graph
We provide a series of graphs for. ..

In [9]:
countAverage = DataFrames.DataFrame()
timeArrayOfTime = TimeArray(ohlcvTime[!, :dates], Float64.(ohlcvTime[!, :TickCount]))
timeArrayOfTick = TimeArray(ohlcvTick[!, :dates], Float64.(ohlcvTick[!, :TickCount]))
timeArrayOfVolume = TimeArray(ohlcvVolume[!, :dates], Float64.(ohlcvVolume[!, :TickCount]))
timeArrayOfDollar = TimeArray(ohlcvDollar[!, :dates], Float64.(ohlcvDollar[!, :TickCount]));

In [10]:
countAverage[!, :Dates] = Date.(timestamp(mean(TimeSeriesResampler.resample(timeArrayOfTime[:A], Dates.Week(1)))))
countAverage[!, :Time] = values(mean(TimeSeriesResampler.resample(timeArrayOfTime[:A], Dates.Week(1))))
countAverage[!, :Tick] = values(mean(TimeSeriesResampler.resample(timeArrayOfTick[:A], Dates.Week(1))))
countAverage[!, :Volume] = values(mean(TimeSeriesResampler.resample(timeArrayOfVolume[:A], Dates.Week(1))))
countAverage[!, :Dollar] = values(mean(TimeSeriesResampler.resample(timeArrayOfDollar[:A], Dates.Week(1))))
countAverage[1:5, :]

,Dates,Time,Tick,Volume,Dollar
,Date,Float64,Float64,Float64,Float64
1,2019-12-30,91.0,84.0,52.6222,48.2857
2,2020-01-06,61.2462,84.0,78.3725,67.6441
3,2020-01-13,62.2154,84.0,95.8571,80.68
4,2020-01-20,61.5577,84.0,88.9444,74.5909
5,2020-01-27,78.2388,84.0,102.765,88.9828


In [11]:
pt = Plots.plot(countAverage[!, :Dates], countAverage[!, :Time], label = "Time", background = :transparent)
pt = plot!(countAverage[!, :Dates],countAverage[!, :Tick], label = "Tick", background = :transparent)
pt = plot!(countAverage[!, :Dates],countAverage[!, :Volume], label = "Volume", background = :transparent)
pt = plot!(countAverage[!, :Dates],countAverage[!, :Dollar], label = "Dollar", background = :transparent)
savefig(pt, "Figs/bar_frequency.png")

In [5]:
# Select a smaller range
dataframe2 = dataframe[(dataframe.dates .>= DateTime(2020, 3, 1) .&& dataframe.dates .<= DateTime(2020, 3, 5)), :]
select!(dataframe2, [:dates, :price, :size])
unique!(dataframe2)
dataframe2.ret = Vector{Union{Float64, Missing}}(log.(dataframe2.price) - log.(circshift(dataframe2.price, 1)))
dataframe2.ret[1] = missing
dropmissing!(dataframe2)
dataframe2.label = sign.(dataframe2.ret)
dataframe2.volumelabeled = dataframe2.label .* dataframe2.size
dataframe2.dollars = dataframe2.volumelabeled .* dataframe2.price
first(dataframe2, 10)

,dates,price,size,ret,label,volumelabeled,dollars
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-03-02T09:00:12,114.7,123.0,8.71878e-5,1.0,123.0,14108.1
2,2020-03-02T09:30:00,115.28,26670.0,0.00504393,1.0,26670.0,3.07452e6
3,2020-03-02T09:30:07,115.34,100.0,0.000520336,1.0,100.0,11534.0
4,2020-03-02T09:30:16,115.22,100.0,-0.00104094,-1.0,-100.0,-11522.0
5,2020-03-02T09:30:26,115.264,172.0,0.000379203,1.0,172.0,19825.4
6,2020-03-02T09:30:27,115.253,557.0,-9.28349e-5,-1.0,-557.0,-64195.9
7,2020-03-02T09:31:00,115.515,238.0,0.00226895,1.0,238.0,27492.5
8,2020-03-02T09:31:02,115.515,148.0,0.0,0.0,0.0,0.0
9,2020-03-02T09:31:03,115.42,100.0,-0.000821011,-1.0,-100.0,-11542.0


In [13]:
ohlcvTime2 = timeBar(dataframe2, 30)
ohlcvTick2 = tickBar(dataframe2, 1000, nothing)
ohlcvVolume2 = volumeBar(dataframe2, 10000, nothing)
ohlcvDollar2 = dollarBar(dataframe2, 500000, nothing);

In [6]:
ohlcvInfo, abs_thetas, thresholds = infoBar(dataframe2, "dollar", 500)
first(ohlcvInfo, 10)

,dates,Open,High,Low,Close,Volume,ValueOfTrades
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-03-02T09:00:12,114.7,115.28,114.7,115.28,26793.0,3.08863e6
2,2020-03-02T09:30:07,115.34,119.78,113.96,119.77,937632.0,1.09618e8
3,2020-03-03T09:30:00,119.72,121.6,115.401,121.12,1.69557e6,2.01469e8
4,2020-03-04T15:52:13,121.22,121.43,121.15,121.32,48431.0,5.87208e6


# Next part

In [7]:
stocks = [:AAPL, :MSFT, :GOOG, :AMZN]
start = DateTime(2020, 1, 1)
allStocks = []
for symb in stocks
    data = DataFrames.DataFrame(yahoo(symb, YahooOpt(period1 = start)))
    priceAdjClose = data.AdjClose
    if getindex(stocks, 1) == symb
        timestamp = data.timestamp
        allStocks = hcat(timestamp, priceAdjClose)
    else
        allStocks = hcat(allStocks, priceAdjClose)
    end
end
allStocksPrices = DataFrames.DataFrame(allStocks, :auto)
allStocksPrices = DataFrames.rename(allStocksPrices, names(allStocksPrices) .=> String.(append!([:Dates], stocks)))
CSV.write("Data_Prices.csv", allStocksPrices)

"Data_Prices.csv"

In [8]:
function pctChange(prices::DataFrames.DataFrame)
    returns = DataFrames.DataFrame()
    for sym in names(prices)[2:end]
        x = prices[!, Symbol(sym)]
        ret = Array{Float64}(undef, length(x))
        ret[1] = NaN
        for i ∈ 2:length(x)
            ret[i] = (x[i]/x[i-1]) - 1
        end
        returns[!, Symbol(sym)] = ret
    end
    return returns
end

pctChange (generic function with 1 method)

In [9]:
allStocksReturns = pctChange(allStocksPrices)[1:end, :]
CSV.write("Data_Returns.csv", allStocksReturns)
allStocksReturns[1:10, :]

,AAPL,MSFT,GOOG,AMZN
,Float64,Float64,Float64,Float64
1,NaN,NaN,NaN,NaN
2,-0.00972184,-0.0124518,-0.00490727,-0.012139
3,0.00796815,0.00258499,0.0246572,0.0148856
4,-0.00470304,-0.00911786,-0.000624052,0.00209162
5,0.0160863,0.0159284,0.00788039,-0.00780864
6,0.021241,0.0124932,0.0110445,0.00479918
7,0.0022605,-0.00462712,0.00697271,-0.0094106
8,0.0213645,0.0120243,0.00664458,0.00432257
9,-0.0135032,-0.00704317,-0.00580175,-0.0115582


# PCA weight

In [10]:
covarinace = cov(Matrix(values(allStocks[:, 2:end])))

4×4 Matrix{Float64}:
 1003.3    1466.51    766.848  419.899
 1466.51   2491.52   1334.09   746.976
  766.848  1334.09    760.354  398.922
  419.899   746.976   398.922  649.596

In [13]:
ω = PCAWeights(covarinace, nothing, 0.1)

4-element Vector{Float64}:
 -0.004149504016806235
  0.010513053367911054
 -0.014572565793935068
 -0.00047937106652688565

# cusum filter

In [14]:
data = allStocksPrices[!, [:Dates, :AAPL]]
first(data, 10)

,Dates,AAPL
,Any,Any
1,2020-01-02,73.6836
2,2020-01-03,72.9672
3,2020-01-06,73.5486
4,2020-01-07,73.2027
5,2020-01-08,74.3803
6,2020-01-09,75.9602
7,2020-01-10,76.1319
8,2020-01-13,77.7584
9,2020-01-14,76.7084


In [15]:
timeEvents = events(data, 5)
first(timeEvents, 10)

10-element Vector{Any}:
 2020-01-17
 2020-02-24
 2020-02-27
 2020-03-02
 2020-03-09
 2020-03-12
 2020-03-13
 2020-03-16
 2020-03-20
 2020-03-24